In [1]:
!pip install pyspark


----
### INCOME

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("landingtoformatted").getOrCreate()

#change the distribucio o  with the accent.
landing_zone = 'landing_zone'
formatted_zone = 'formatted_zone'

datasets = [
    "landing_zone/income/2014_Distribucio_territorial_renda_familiar.csv",
    "landing_zone/income/2015_Distribucio_territorial_renda_familiar.csv",
    "landing_zone/income/2016_Distribucio_territorial_renda_familiar.csv",
    "landing_zone/income/2017_Distribucio_territorial_renda_familiar.csv"
]
income_df = spark.read.csv(datasets[0], header=True, inferSchema=True,nullValue ="-")

for dataset in datasets[1:]:
    df_next = spark.read.csv(dataset, header=True, inferSchema=True, nullValue ="-")
    income_df = income_df.union(df_next)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/06/18 18:56:43 WARN Utils: Your hostname, MacBook-Air-de-Pol.local, resolves to a loopback address: 127.0.0.1; using 10.60.217.176 instead (on interface en0)
25/06/18 18:56:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/18 18:56:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/06/18 18:56:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
income_df.show(100)

+----+--------------+-------------------+----------+--------------------+--------+-------------------------+
| Any|Codi_Districte|      Nom_Districte|Codi_Barri|           Nom_Barri|Població|Índex RFD Barcelona = 100|
+----+--------------+-------------------+----------+--------------------+--------+-------------------------+
|2014|             1|       Ciutat Vella|         1|            el Raval|   48471|                     65.9|
|2014|             1|       Ciutat Vella|         2|      el Barri Gòtic|   15911|                     98.5|
|2014|             1|       Ciutat Vella|         3|      la Barceloneta|   15181|                     84.5|
|2014|             1|       Ciutat Vella|         4|Sant Pere, Santa ...|   22674|                     92.5|
|2014|             2|           Eixample|         5|       el Fort Pienc|   31785|                    104.5|
|2014|             2|           Eixample|         6|  la Sagrada Família|   51562|                     92.4|
|2014|             

Standarize, eliminate the NULL columns

In [3]:
#we filter the data to remove rows with null values in the 'Índex RFD Barcelona = 100' column or the Names
from pyspark.sql.functions import col

filt_income = income_df.filter(
    ~(
        col('`Índex RFD Barcelona = 100`').isNull() |
        (col('Nom_districte') == 'No consta')
     )
)

----
### Incidencies

In [12]:
datasets = [
    "landing_zone/incidences/2014_Peticions_ciutadanes.csv",
    "landing_zone/incidences/2015_Peticions_ciutadanes.csv",
    "landing_zone/incidences/2016_Peticions_ciutadanes.csv",
    "landing_zone/incidences/2017_Peticions_ciutadanes.csv"
]
incidences_df = spark.read.csv(datasets[0], header=True, inferSchema=True)
for dataset in datasets[1:]:
    df_next = spark.read.csv(dataset, header=True, inferSchema=True)
    incidences_df = incidences_df.union(df_next)

In [13]:
incidences_df.show(100)

+--------------+-------------------+------------------+--------------------+--------------------+--------+-----------+--------------------+------------+------------+-----------+-------------+---------------+------+--------------------+-------------+------------------+---------+-------------+-------------+-------+--------------------+----------+-----+-----------------+------------------+
|CODI_DISTRICTE|          DISTRICTE|ANY_DATA_TANCAMENT|              DETALL|               BARRI|FITXA_ID|    LATITUD|                AREA|COORDENADA_Y|COORDENADA_X|   LONGITUD|DIA_DATA_ALTA|CANALS_RESPOSTA|NUMERO|             ELEMENT|SECCIO_CENSAL|MES_DATA_TANCAMENT|TIPUS_VIA|ANY_DATA_ALTA|MES_DATA_ALTA| SUPORT|              CARRER|CODI_BARRI|  _id|            TIPUS|DIA_DATA_TANCAMENT|
+--------------+-------------------+------------------+--------------------+--------------------+--------+-----------+--------------------+------------+------------+-----------+-------------+---------------+------+------

In [14]:
from pyspark.sql.functions import col, sum
incidences_df.select([sum(col(c).isNull().cast("int")).alias(c) for c in incidences_df.columns]).show()

+--------------+---------+------------------+------+-----+--------+-------+----+------------+------------+--------+-------------+---------------+------+-------+-------------+------------------+---------+-------------+-------------+------+------+----------+---+-----+------------------+
|CODI_DISTRICTE|DISTRICTE|ANY_DATA_TANCAMENT|DETALL|BARRI|FITXA_ID|LATITUD|AREA|COORDENADA_Y|COORDENADA_X|LONGITUD|DIA_DATA_ALTA|CANALS_RESPOSTA|NUMERO|ELEMENT|SECCIO_CENSAL|MES_DATA_TANCAMENT|TIPUS_VIA|ANY_DATA_ALTA|MES_DATA_ALTA|SUPORT|CARRER|CODI_BARRI|_id|TIPUS|DIA_DATA_TANCAMENT|
+--------------+---------+------------------+------+-----+--------+-------+----+------------+------------+--------+-------------+---------------+------+-------+-------------+------------------+---------+-------------+-------------+------+------+----------+---+-----+------------------+
|           123|      123|                 0|     0|  123|       0|    142|   0|         142|         142|     142|            0|             

In [15]:
filt_incidences = incidences_df.filter(~(col('CODI_DISTRICTE').isNull() & col('DISTRICTE').isNull()))

In [16]:
# we need to rename the columns to match the income_df
# We change the column names ANY_DATA_ALTA, to ANY so we can match later the year where the incident happened.
filt_incidences = (filt_incidences.withColumnRenamed('CODI_BARRI', 'Codi_Barri')
        .withColumnRenamed('CODI_DISTRICTE', 'Codi_Districte').withColumnRenamed('ANY_DATA_ALTA', 'Any'))

with renaming the columns we facilitate the later the unification of the tables

In [17]:
filt_incidences.show(100)

+--------------+-------------------+------------------+--------------------+--------------------+--------+-----------+--------------------+------------+------------+-----------+-------------+---------------+------+--------------------+-------------+------------------+---------+----+-------------+--------------+--------------------+----------+---+-----------+------------------+
|Codi_Districte|          DISTRICTE|ANY_DATA_TANCAMENT|              DETALL|               BARRI|FITXA_ID|    LATITUD|                AREA|COORDENADA_Y|COORDENADA_X|   LONGITUD|DIA_DATA_ALTA|CANALS_RESPOSTA|NUMERO|             ELEMENT|SECCIO_CENSAL|MES_DATA_TANCAMENT|TIPUS_VIA| Any|MES_DATA_ALTA|        SUPORT|              CARRER|Codi_Barri|_id|      TIPUS|DIA_DATA_TANCAMENT|
+--------------+-------------------+------------------+--------------------+--------------------+--------+-----------+--------------------+------------+------------+-----------+-------------+---------------+------+--------------------+-----

-----
### Population

In [18]:
datasets = [
    "landing_zone/population_by_geographical/2014_pad_mdb_nacionalitat-regio_sexe.json",
    "landing_zone/population_by_geographical/2015_pad_mdb_nacionalitat-regio_sexe.json",
    "landing_zone/population_by_geographical/2016_pad_mdb_nacionalitat-regio_sexe.json",
    "landing_zone/population_by_geographical/2017_pad_mdb_nacionalitat-regio_sexe.json"
]
population_df = spark.read.json(datasets[0])
for dataset in datasets[1:]:
    df_next = spark.read.json(dataset)
    population_df = population_df.union(df_next)

In [19]:
population_df.show(100)

+----------+--------------+---------------+------------------+--------------+-------------+----+-----+
|Codi_Barri|Codi_Districte|Data_Referencia|NACIONALITAT_REGIO|     Nom_Barri|Nom_Districte|SEXE|Valor|
+----------+--------------+---------------+------------------+--------------+-------------+----+-----+
|         1|             1|     2014-01-01|                 1|      el Raval| Ciutat Vella|   1|   ..|
|         1|             1|     2014-01-01|                 1|      el Raval| Ciutat Vella|   2|   ..|
|         1|             1|     2014-01-01|                 2|      el Raval| Ciutat Vella|   1|    6|
|         1|             1|     2014-01-01|                 2|      el Raval| Ciutat Vella|   2|    8|
|         1|             1|     2014-01-01|                 3|      el Raval| Ciutat Vella|   1|  682|
|         1|             1|     2014-01-01|                 3|      el Raval| Ciutat Vella|   2|  989|
|         1|             1|     2014-01-01|                 4|      el Ra

In [20]:
from pyspark.sql import functions as F
population_df = population_df.withColumn("Any", F.substring("Data_Referencia", 1, 4).cast("int"))

In [21]:
import pandas as pd
csv = pd.read_csv('landing_zone/pad_dimensions.csv')

In [22]:
csv

,Codi_Dimensio,Desc_Dimensio,Codi_Valor,Desc_Valor_CA,Desc_Valor_ES,Desc_Valor_EN
0,1,SEXE,1,Dona,Mujer,Female
1,1,SEXE,2,Home,Hombre,Male
2,2,EDAT_1,0,0 anys,0 años,0 years
3,2,EDAT_1,1,1 anys,1 años,1 years
4,2,EDAT_1,2,2 anys,2 años,2 years
...,...,...,...,...,...,...
1138,26,NACIONALITAT_DEST,142,Maurici,Mauricio,Mauritius
1139,26,NACIONALITAT_DEST,144,"Iraq, l'",Iraq,Iraq
1140,26,NACIONALITAT_DEST,145,Kenya,Kenia,Kenya
1141,26,NACIONALITAT_DEST,146,Seychelles,Seychelles,Seychelles


In [23]:
population_df.columns

['Codi_Barri',
 'Codi_Districte',
 'Data_Referencia',
 'NACIONALITAT_REGIO',
 'Nom_Barri',
 'Nom_Districte',
 'SEXE',
 'Valor',
 'Any']

In [24]:
csv = spark.createDataFrame(csv)

We merged the two datasets so that, instead of having encoded values in the 'SEXE' or 'NACIONALITAT_REGIO' columns, we now have the actual values. To do this, we had to download an additional dataset that contained both the actual values and their corresponding encoding numbers.

In [25]:
from pyspark.sql.functions import col, when
dim_names = [row['Desc_Dimensio'] for row in csv.select('Desc_Dimensio').distinct().collect()]

for aa in population_df.columns:
    if aa in dim_names:
        mapping_rows = csv.filter(col('Desc_Dimensio') == aa).select('Codi_Valor', 'Desc_Valor_EN').collect()
        mapping_dict = {int(row['Codi_Valor']): row['Desc_Valor_EN'] for row in mapping_rows if row['Codi_Valor'] is not None}
        expr = None
        for code, label in mapping_dict.items():
            condition = (col(aa) == code)
            expr = when(condition, label) if expr is None else expr.when(condition, label)

        expr = expr.otherwise(col(aa))
        population_df = population_df.withColumn(aa, expr)

MEL: this gaves me an error idk how did you do it.

In [26]:
population_df.show(100)

25/06/18 19:01:46 ERROR Executor: Exception in task 0.0 in stage 52.0 (TID 76)
org.apache.spark.SparkNumberFormatException: [CAST_INVALID_INPUT] The value 'Eastern Africa' of the type "STRING" cannot be cast to "BIGINT" because it is malformed. Correct the value as per the syntax, or change its target type. Use `try_cast` to tolerate malformed input and return NULL instead. SQLSTATE: 22018
== DataFrame ==
"otherwise" was called from
line 13 in cell [25]

	at org.apache.spark.sql.errors.QueryExecutionErrors$.invalidInputInCastToNumberError(QueryExecutionErrors.scala:145)
	at org.apache.spark.sql.catalyst.util.UTF8StringUtils$.withException(UTF8StringUtils.scala:51)
	at org.apache.spark.sql.catalyst.util.UTF8StringUtils$.toLongExact(UTF8StringUtils.scala:31)
	at org.apache.spark.sql.catalyst.util.UTF8StringUtils.toLongExact(UTF8StringUtils.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark

NumberFormatException: [CAST_INVALID_INPUT] The value 'Eastern Africa' of the type "STRING" cannot be cast to "BIGINT" because it is malformed. Correct the value as per the syntax, or change its target type. Use `try_cast` to tolerate malformed input and return NULL instead. SQLSTATE: 22018
== DataFrame ==
"otherwise" was called from
line 13 in cell [25]


### Before this we have to partition the tables in the more optimal way for our analysis.

I think we should make the partition by years.

In [27]:
filt_incidences.write.mode('overwrite').parquet("formatted_zone/incidences")
filt_income.write.mode('overwrite').parquet("formatted_zone/income")
population_df.write.mode('overwrite').parquet("formatted_zone/population_by_geographical")


25/06/18 19:01:50 ERROR Utils: Aborting task
org.apache.spark.SparkNumberFormatException: [CAST_INVALID_INPUT] The value 'Eastern Africa' of the type "STRING" cannot be cast to "BIGINT" because it is malformed. Correct the value as per the syntax, or change its target type. Use `try_cast` to tolerate malformed input and return NULL instead. SQLSTATE: 22018
== DataFrame ==
"otherwise" was called from
line 13 in cell [25]

	at org.apache.spark.sql.errors.QueryExecutionErrors$.invalidInputInCastToNumberError(QueryExecutionErrors.scala:145)
	at org.apache.spark.sql.catalyst.util.UTF8StringUtils$.withException(UTF8StringUtils.scala:51)
	at org.apache.spark.sql.catalyst.util.UTF8StringUtils$.toLongExact(UTF8StringUtils.scala:31)
	at org.apache.spark.sql.catalyst.util.UTF8StringUtils.toLongExact(UTF8StringUtils.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage2.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator

NumberFormatException: [CAST_INVALID_INPUT] The value 'Eastern Africa' of the type "STRING" cannot be cast to "BIGINT" because it is malformed. Correct the value as per the syntax, or change its target type. Use `try_cast` to tolerate malformed input and return NULL instead. SQLSTATE: 22018
== DataFrame ==
"otherwise" was called from
line 13 in cell [25]
